In [8]:
# --- Installation Commands (Run this once in a fresh Jupyter cell) ---
# IMPORTANT: Run these commands at the very beginning of your notebook
# or in your terminal (after activating your conda env if applicable).
# After successful installation, you might need to restart your Jupyter kernel
# and run all cells from the beginning.
#
#!pip install langchain-openai
#!pip install requests
#!pip install chromadb
#!pip install langchain
#!pip install langchain-community
!pip install pypdf

In [2]:
import os
import shutil
import requests
import json

# LangChain Imports
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.embeddings import Embeddings
from langchain_community.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

In [3]:
# Custom Embeddings Class for Nutanix Granite Embedding API
# This class wraps your API endpoint to work seamlessly with LangChain's embedding interface.
class NutanixGraniteEmbeddings(Embeddings):
    """
    Custom Embeddings class to interact with the Nutanix AI Granite Embedding endpoint.
    It extends LangChain's Embeddings base class to provide custom embedding logic.
    """
    def __init__(self, api_key: str, endpoint_url: str, model_name: str):
        self.api_key = api_key
        self.endpoint_url = endpoint_url
        self.model_name = model_name
        self.session = requests.Session()

    def embed_documents(self, texts: list[str]) -> list[list[float]]:
        """
        Embeds a list of documents.
        """
        return self._get_embeddings(texts)

    def embed_query(self, text: str) -> list[float]:
        """
        Embeds a single query string.
        """
        return self._get_embeddings([text])[0]

    def _get_embeddings(self, texts: list[str]) -> list[list[float]]:
        """
        Internal helper method to make the actual API call.
        """
        headers = {
            "Authorization": f"Bearer {self.api_key}",
            "accept": "application/json",
            "Content-Type": "application/json"
        }
        payload = {
            "model": self.model_name,
            "input": texts,
            "encoding_format": "float"
        }

        response = None
        try:
            response = self.session.post(self.endpoint_url, headers=headers, json=payload)
            response.raise_for_status()
            result = response.json()

            if "data" in result and len(result["data"]) > 0:
                embeddings = [item["embedding"] for item in result["data"]]
                return embeddings
            else:
                raise ValueError("API response did not contain expected 'data' or embeddings.")

        except requests.exceptions.RequestException as e:
            print(f"API request failed: {e}")
            if response is not None:
                print(f"Response status: {response.status_code}")
                print(f"Response text: {response.text}")
            raise
        except json.JSONDecodeError:
            print(f"Failed to decode JSON from API response: {e}")
            raise
        except Exception as e:
            print(f"An unexpected error occurred during embedding: {e}")
            raise


In [2]:
os.environ["NUTANIX_API_KEY"] = "db076c5a-b4d9-44b5-bc93-0be3922df085"

In [4]:
# --- Configuration ---
PDF_DIR = "data/pdfs"
CHROMA_DB_DIR = "my_vector_db" # Directory for ChromaDB persistence

# Nutanix AI API Configuration
EMBEDDING_API_MODEL_NAME = "embedcpu"
NUTANIX_GRANITE_EMBEDDING_ENDPOINT = "https://ai.nutanix.com/api/v1/embeddings"
LLM_API_MODEL_NAME = "mobileyedemo"
NUTANIX_LLM_ENDPOINT = "https://ai.nutanix.com/api/v1" # Corrected base URL for ChatOpenAI

# IMPORTANT: Ensure NUTANIX_API_KEY environment variable is set.
NUTANIX_API_KEY = os.getenv("NUTANIX_API_KEY")

if not NUTANIX_API_KEY:
    print("WARNING: NUTANIX_API_KEY environment variable not set.")
    print("Please set this environment variable for the Nutanix AI APIs to work.")
    exit("API Key for Nutanix AI not found. Exiting.")

In [6]:
# --- 1. Prepare Directories ---
print("\n--- 1. Preparing Directories ---")
os.makedirs(PDF_DIR, exist_ok=True)
print(f"Ensuring PDF directory exists: {PDF_DIR}")

# IMPORTANT: Due to potential Windows PermissionError, it's recommended to MANUALLY
# delete the 'my_vector_db' folder if you encounter issues, or change CHROMA_DB_DIR
# if you want to regenerate the database.
if os.path.exists(CHROMA_DB_DIR):
    try:
        print(f"Attempting to remove existing ChromaDB data from: {CHROMA_DB_DIR}")
        shutil.rmtree(CHROMA_DB_DIR)
        print("Existing ChromaDB data removed successfully.")
    except PermissionError as e:
        print(f"PermissionError: Could not remove {CHROMA_DB_DIR}. Please close any programs accessing it and delete it manually.")
        print(f"Error details: {e}")
        exit("Cannot proceed without clearing old database. Please delete 'my_vector_db' manually or change CHROMA_DB_DIR.")
    except Exception as e:
        print(f"An unexpected error occurred while trying to remove {CHROMA_DB_DIR}: {e}")
        exit("Cannot proceed due to database cleanup error.")

os.makedirs(CHROMA_DB_DIR, exist_ok=True)
print(f"Ensuring ChromaDB directory exists: {CHROMA_DB_DIR}")


--- 1. Preparing Directories ---
Ensuring PDF directory exists: data/pdfs
Ensuring ChromaDB directory exists: my_vector_db


In [9]:
# --- 2. Load PDF Documents ---
print(f"\n--- 2. Loading PDF Documents from '{PDF_DIR}' ---")
documents = []
for filename in os.listdir(PDF_DIR):
    if filename.endswith(".pdf"):
        filepath = os.path.join(PDF_DIR, filename)
        try:
            loader = PyPDFLoader(filepath)
            documents.extend(loader.load())
            print(f"Loaded '{filename}'")
        except Exception as e:
            print(f"Error loading '{filename}': {e}")

if not documents:
    print(f"No PDF documents found in '{PDF_DIR}'. Please place some PDFs there.")
    print("Example: You can create a dummy PDF file for testing or download one.")
    exit("No PDFs found. Exiting.")



--- 2. Loading PDF Documents from 'data/pdfs' ---
Loaded '03_Wills_and_Directives.pdf'
Loaded '1-1-A.pdf'
Loaded '1-2-a.pdf'
Loaded '1095BHealthCoverageTaxDocument.pdf'
Loaded '1200-13-01.20230419.pdf'
Loaded '1200-13-02.20221004.pdf'
Loaded '1200-13-03.pdf'
Loaded '1200-13-04.20090719.pdf'
Loaded '1200-13-05.20151220.pdf'
Loaded '1200-13-06.pdf'
Loaded '1200-13-07.pdf'
Loaded '1200-13-08.pdf'
Loaded '1200-13-09.pdf'
Loaded '1200-13-10.pdf'
Loaded '1200-13-11.20190403.pdf'
Loaded '1200-13-12.pdf'
Loaded '1200-13-13.20190403.pdf'
Loaded '1200-13-13.20250202.pdf'
Loaded '1200-13-14.20250202.pdf'
Loaded '1200-13-15.20100829.pdf'
Loaded '1200-13-16.20111128.pdf'
Loaded '1200-13-17.20131113.pdf'
Loaded '1200-13-18.20130415.pdf'
Loaded '1200-13-19.20150503.pdf'
Loaded '1200-13-20.20220811.pdf'
Loaded '1200-13-21.20250202.pdf'
Loaded '1915cWaiverAmendmentsPlainLanguageOverview.pdf'
Loaded '2-1-a.pdf'
Loaded '2-2-a.pdf'
Loaded '2-6-a.pdf'
Loaded '2011-SocialSecurityNumberPrivacy-TN.pdf'
Loade

invalid pdf header: b'<!DOC'
EOF marker not found
invalid pdf header: b'<!DOC'
EOF marker not found


Loaded 'Building_Strong_BrainsOVERVIEW-MISSION.pdf'
Error loading 'CFSRManual.pdf': Stream has ended unexpectedly
Error loading 'CFSR_Roadshow.pdf': Stream has ended unexpectedly
Loaded 'ChangeTennCareIIIProgramMaternalHealthEnhancements.pdf'
Loaded 'CHIP2101Group.pdf'
Loaded 'CHIPEligibilityProcessing.pdf'
Loaded 'CHIPMAGIEligibility.pdf'
Loaded 'CHIPNonFinancialEligibility.pdf'
Loaded 'CHIPStatePlan.pdf'
Loaded 'CHIPTitleXXIExpansion.pdf'
Loaded 'ChiropracticSPAPublicNotice.pdf'
Loaded 'CHOICESAtRiskDemonstrationGroup.pdf'
Loaded 'CHOICESBenefitsComprehensiveNotice.pdf'
Loaded 'CHOICESGroup3ComprehensiveNotice.pdf'
Loaded 'CivilRightsNotice.pdf'
Loaded 'ClinicalTrialsSPAPublicNotice.pdf'
Loaded 'CMSApprovedEvaluationDesign.pdf'
Loaded 'CMSApprovedEvaluationDesignHCBS.pdf'
Loaded 'CMSApprovedEvaluationDesignRiskMitigation.pdf'
Loaded 'CMSApprovedHCBSPHE.pdf'
Loaded 'CMSApprovedInterimEvaluationReport.pdf'
Loaded 'CMSApprovedPHERiskMitigationt.pdf'
Loaded 'CMSAR040722.pdf'
Loaded 'Comp

Ignoring wrong pointing object 17 0 (offset 0)
Ignoring wrong pointing object 60 0 (offset 0)
Ignoring wrong pointing object 62 0 (offset 0)
Ignoring wrong pointing object 67 0 (offset 0)
Ignoring wrong pointing object 69 0 (offset 0)
Ignoring wrong pointing object 71 0 (offset 0)
Ignoring wrong pointing object 73 0 (offset 0)
Ignoring wrong pointing object 77 0 (offset 0)
Ignoring wrong pointing object 82 0 (offset 0)
Ignoring wrong pointing object 84 0 (offset 0)
Ignoring wrong pointing object 86 0 (offset 0)
Ignoring wrong pointing object 88 0 (offset 0)
Ignoring wrong pointing object 90 0 (offset 0)
Ignoring wrong pointing object 92 0 (offset 0)
Ignoring wrong pointing object 95 0 (offset 0)
Ignoring wrong pointing object 101 0 (offset 0)


Loaded 'TN-Tenncare-Transitions-Book3.pdf'


Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 53 0 (offset 0)
Ignoring wrong pointing object 55 0 (offset 0)
Ignoring wrong pointing object 57 0 (offset 0)
Ignoring wrong pointing object 59 0 (offset 0)
Ignoring wrong pointing object 63 0 (offset 0)
Ignoring wrong pointing object 68 0 (offset 0)
Ignoring wrong pointing object 70 0 (offset 0)
Ignoring wrong pointing object 72 0 (offset 0)
Ignoring wrong pointing object 74 0 (offset 0)
Ignoring wrong pointing object 76 0 (offset 0)
Ignoring wrong pointing object 82 0 (offset 0)


Loaded 'TN-Tenncare-Transitions-Book4.pdf'
Loaded 'TN-Tenncare-Transitions-Book5.pdf'
Loaded 'TN-TransplantServices.pdf'
Loaded 'TN-Wellness-Visits-Flyer.pdf'
Loaded 'TNAdvance-Directives-Living-Wills-AR.pdf'
Loaded 'TNCare_Flier.pdf'
Loaded 'TN_2020%202024_CFSP_Approval_Letter_signed.pdf'
Loaded 'TN_Child_Family_Service_Plan_2025-2029.pdf'
Loaded 'TN_Civil_Rights_Notice.pdf'
Loaded 'toc.pdf'
Loaded 'UCPoolsComprehensiveNotice.pdf'
Loaded 'UCPoolsComprehensiveNotice3.pdf'
Loaded 'WellnessRoadMapAgeSevenUp.pdf'
Loaded 'WellnessRoadMapAgeZeroToSix.pdf'
Loaded 'WellnessRoadMapFull.pdf'


In [10]:
# --- 3. Split Documents into Chunks ---
print("\n--- 3. Splitting Documents into Chunks ---")
# Adjusted chunk_size and chunk_overlap for the Granite embedding model's 512 token limit.
# 400 characters is approximately 100-150 tokens, providing a safe buffer.
text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)
print(f"Original documents loaded: {len(documents)}")
print(f"Documents split into {len(chunks)} chunks.")
if chunks:
    print(f"First chunk example (first 200 chars):\n'{chunks[0].page_content[:200]}'")
else:
    exit("No chunks generated from documents. Exiting.")




--- 3. Splitting Documents into Chunks ---
Original documents loaded: 28084
Documents split into 229160 chunks.
First chunk example (first 200 chars):
'LIVING WILLS
A living will, originally designed to be completed when a person was diagnosed with a terminal illness, is an 
earlier version of an advance care directive document that tells a doctor ho'


In [11]:
# --- 4. Create Embeddings Model Instance (Nutanix Granite API) ---
print(f"\n--- 4. Creating Embeddings using Nutanix Granite Embedding API ('{EMBEDDING_API_MODEL_NAME}') ---")
try:
    embeddings_model = NutanixGraniteEmbeddings(
        api_key=NUTANIX_API_KEY,
        endpoint_url=NUTANIX_GRANITE_EMBEDDING_ENDPOINT,
        model_name=EMBEDDING_API_MODEL_NAME
    )
    # Test a dummy embedding to ensure API connection works and get dimension
    dummy_embedding = embeddings_model.embed_query("test connection")
    print(f"Nutanix Granite Embedding API connected successfully. Embedding dimension: {len(dummy_embedding)}")
except Exception as e:
    print(f"Error initializing or testing Nutanix Granite Embedding API: {e}")
    print("Please check your NUTANIX_API_KEY, endpoint URL, and internet connection.")
    exit()


--- 4. Creating Embeddings using Nutanix Granite Embedding API ('embedcpu') ---
Nutanix Granite Embedding API connected successfully. Embedding dimension: 384


In [12]:
# --- 5. Create and Populate the Vector Database (ChromaDB) ---
print("\n--- 5. Creating and Populating the Vector Database (ChromaDB) ---")
try:
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings_model,
        persist_directory=CHROMA_DB_DIR
    )
    vectorstore.persist()
    print(f"Vector database created and populated with {len(chunks)} chunks.")
    print(f"ChromaDB persisted to '{CHROMA_DB_DIR}'")

    # --- Optional: Test the Vector Database ---
    print("\n--- Optional: Testing Vector Database with a Similarity Search ---")
    query = "What is the main topic of these documents?"
    docs_found = vectorstore.similarity_search(query, k=2)
    print(f"Query: '{query}'")
    print("\nTop 2 relevant chunks found:")
    for i, doc in enumerate(docs_found):
        print(f"--- Chunk {i+1} (Source: {doc.metadata.get('source', 'N/A')}, Page: {doc.metadata.get('page', 'N/A')}) ---")
        print(doc.page_content[:300] + "...")
        print("-" * 30)

except Exception as e:
    print(f"Error creating/populating ChromaDB: {e}")
    exit("Failed to create or populate the vector database.")



--- 5. Creating and Populating the Vector Database (ChromaDB) ---


C:\Users\JoshStevenson\AppData\Local\Temp\ipykernel_3540\3102586629.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


Vector database created and populated with 229160 chunks.
ChromaDB persisted to 'my_vector_db'

--- Optional: Testing Vector Database with a Similarity Search ---
Query: 'What is the main topic of these documents?'

Top 2 relevant chunks found:
--- Chunk 1 (Source: data/pdfs\TN-MemberHandbook-EN.pdf, Page: 185) ---
These papers must be signed, and either witnessed or notarized. Once the papers are signed by 
everyone, it is your rule. It stays like this unless you change your mind.
Table of contents...
------------------------------
--- Chunk 2 (Source: data/pdfs\Public-Opinion-Report-ACES.pdf, Page: 2) ---
ii 
 
Table of Contents 
Purpose and Methodology .......................................................................................................................... 1 
Findings and Discussion ........................................................................................................
------------------------------


In [13]:
# --- 6. Initialize the Remote LLM (Nutanix AI Endpoint) ---
print(f"\n--- 6. Initializing the Remote Language Model (Nutanix AI Endpoint: '{LLM_API_MODEL_NAME}') ---")
try:
    llm = ChatOpenAI(
        model=LLM_API_MODEL_NAME,
        temperature=0.0, # For more deterministic answers
        openai_api_key=NUTANIX_API_KEY,
        base_url=NUTANIX_LLM_ENDPOINT # ChatOpenAI will append /chat/completions to this base_url
    )
    print(f"Remote LLM '{llm.model_name}' initialized successfully via Nutanix AI Endpoint.")
except Exception as e:
    print(f"Error initializing remote LLM: {e}")
    print("Please ensure:")
    print("1. Your NUTANIX_API_KEY is correctly set.")
    print(f"2. The base endpoint URL '{NUTANIX_LLM_ENDPOINT}' is correct and accessible.")
    print(f"3. The model '{LLM_API_MODEL_NAME}' is available at the endpoint.")
    exit("Failed to initialize LLM. Exiting.")


--- 6. Initializing the Remote Language Model (Nutanix AI Endpoint: 'mobileyedemo') ---
Remote LLM 'mobileyedemo' initialized successfully via Nutanix AI Endpoint.


In [14]:
# --- 7. Create Retriever and Build RAG Chain ---
print("\n--- 7. Creating Retriever and Building RAG Chain ---")
retriever = vectorstore.as_retriever(search_kwargs={"k": 3}) # Retrieve top 3 most relevant chunks
prompt = ChatPromptTemplate.from_messages([
    # Updated system prompt to explicitly request English output
    ("system", "Answer the user's question based on the provided context only. Respond concisely and in English.\nContext: {context}"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, document_chain)
print("RAG chain built successfully.")


--- 7. Creating Retriever and Building RAG Chain ---
RAG chain built successfully.


In [25]:
# --- 8. Perform Interactive Inference ---
print("\n--- 8. Performing Interactive Inference with RAG ---")
print("Enter your questions below. Type 'exit' or 'quit' to end the session.")

while True:
    user_query = input("\nYour Question: ")
    if user_query.lower() in ["exit", "quit"]:
        print("Exiting interactive session. Goodbye!")
        break

    try:
        # Invoke the RAG chain with your query
        response = rag_chain.invoke({"input": user_query})
        print("\nLLM Answer:")
        print(response["answer"])

        print("\nRetrieved Context (Documents used by LLM):")
        if response["context"]:
            for j, doc in enumerate(response["context"]):
                print(f"  - Document {j+1} (Source: {doc.metadata.get('source', 'N/A')}, Page: {doc.metadata.get('page', 'N/A')})")
                print(f"    Content (first 200 chars): {doc.page_content[:200]}...")
        else:
            print("  No relevant documents were retrieved.")

    except Exception as e:
        print(f"Error during inference for query '{user_query}': {e}")
        print("Please ensure your Nutanix API key is valid and the LLM endpoint is accessible.")
        break # Breaking on error for interactive stability

print("\nConsolidated RAG Workflow Complete. Ready for your application!")


--- 8. Performing Interactive Inference with RAG ---
Enter your questions below. Type 'exit' or 'quit' to end the session.

LLM Answer:
To find a doctor who takes TennCare insurance, you can:

1. Visit the TennCare website ([www.tn.gov/tenncare](http://www.tn.gov/tenncare)) and click on the "Find a Provider" tab.
2. Contact your local TennCare office or call 1-800-449-3866 (Monday-Friday, 8:00 AM - 6:00 PM EST) for assistance finding a provider in your area.
3. Ask your primary care physician or other healthcare provider if they participate in the TennCare program.
4. Check with your local hospital or healthcare system to see if they have a list of providers participating in TennCare.
5. You can also search online for "TennCare providers" along with your city or zip code to find a list of participating providers in your area.

Retrieved Context (Documents used by LLM):
  - Document 1 (Source: data/pdfs\pro08001.pdf, Page: 3)
    Content (first 200 chars): TennCare for services that ar

In [15]:
!pip install streamlit

In [16]:
import sys
import os

# Get the path to the Python executable used by this Jupyter kernel
jupyter_python_executable = sys.executable

# Construct the likely path to the streamlit.exe script for this Python installation
# This is typically in a 'Scripts' subfolder next to the python.exe
streamlit_exe_path = os.path.join(
    os.path.dirname(jupyter_python_executable),
    "Scripts",
    "streamlit.exe"
)

# Fallback for common user-installed Python locations if the first path doesn't exist
if not os.path.exists(streamlit_exe_path):
    appdata_local_packages_path = os.path.join(
        os.path.expanduser("~"),
        "AppData", "Local", "Packages"
    )
    for root, dirs, files in os.walk(appdata_local_packages_path):
        if "Python" in root and "Scripts" in root and "streamlit.exe" in files:
            streamlit_exe_path = os.path.join(root, "streamlit.exe")
            break
    else:
        streamlit_exe_path = None # Not found in common paths

if streamlit_exe_path and os.path.exists(streamlit_exe_path):
    print(f"**FOUND STREAMLIT EXECUTABLE:**")
    print(f"  {streamlit_exe_path}")
else:
    print("ERROR: Could not find 'streamlit.exe'. Please ensure Streamlit is installed.")
    print("You might need to re-run the `!pip install streamlit` command in a fresh kernel.")

**FOUND STREAMLIT EXECUTABLE:**
  c:\Users\JoshStevenson\miniconda3\envs\rag_nutanix_env\Scripts\streamlit.exe


In [17]:
import sys
import os
streamlit_exe_path = os.path.join(os.path.dirname(sys.executable), "Scripts", "streamlit.exe")
if not os.path.exists(streamlit_exe_path):
    # Fallback for common user-installed Python locations
    local_packages_dir = os.path.join(
        os.path.expanduser("~"), "AppData", "Local", "Packages",
        "PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0",
        "LocalCache", "local-packages", "Python312", "Scripts", "streamlit.exe"
    )
    if os.path.exists(local_packages_dir):
        streamlit_exe_path = local_packages_dir
    else:
        streamlit_exe_path = None
if streamlit_exe_path and os.path.exists(streamlit_exe_path):
    print(streamlit_exe_path)
else:
    print("Error: streamlit.exe path not found.")

c:\Users\JoshStevenson\miniconda3\envs\rag_nutanix_env\Scripts\streamlit.exe
